In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import os
import gc

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import spacy

import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
%matplotlib inline
import os
import gc

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
from sklearn.manifold import TSNE

from os import path
from PIL import Image

In [ ]:
drug_train_data = pd.read_csv(r"C:\Users\user\OneDrive\Documents\Neuranceai\dataset\train.csv")
drug_train_data.head(2)  

In [ ]:
drug_train_data.loc[:, ["effectiveness_rating", "base_score"]]

In [ ]:
print("Number of data points:",drug_train_data.shape[0])

# no of records in a dataset  
print("Number of Attributes :",drug_train_data.shape[1])

Means Our dataset has 32165 record

and 8 Columns

In Which base_score is our Target variable

In [ ]:
# sort on the basis of "drug_approved_by_UIC" coulmn
drug_train_data = drug_train_data.sort_values('drug_approved_by_UIC')
drug_train_data.loc[:, "drug_approved_by_UIC"]

In [ ]:
drug_train_data.info()

In [ ]:
sns.kdeplot(drug_train_data["effectiveness_rating"], color="#7C3794", label='effectiveness_rating')
sns.kdeplot(drug_train_data["base_score"], color="#00ff40", label='base_score')
plt.legend()

In [ ]:
sns.kdeplot?

In [ ]:
sns.histplot(drug_train_data["number_of_times_prescribed"], color="#7C3794")

In [ ]:
drug_train_data["number_of_times_prescribed"]

In [ ]:
plt.scatter(drug_train_data["base_score"], drug_train_data["number_of_times_prescribed"])

In [ ]:
sns.set_style("whitegrid")
sns.pairplot(data = drug_train_data[["effectiveness_rating", "number_of_times_prescribed", "base_score"]], kind="scatter")
plt.show()

In [ ]:
# sns.set_style("whitegrid")
# sns.pairplot(data = drug_train_data[["effectiveness_rating", "number_of_times_prescribed", "base_score"]], kind="kde")
# plt.show()

In [ ]:
qids = pd.Series(drug_train_data['name_of_drug'].tolist())
unique_qs = len(np.unique(qids))
unique_qs

In [ ]:
name_of_drug = pd.Series(drug_train_data['name_of_drug'].tolist())
unique_drugs = len(np.unique(name_of_drug))
print ('Total number of  Unique Questions are: {}\n'.format(unique_drugs))

print ('Max number of times a single question is repeated: {}\n'.format(max(name_of_drug.value_counts()))) 

drug_vals=name_of_drug.value_counts()
print(drug_vals)

drug_vals=drug_vals.values

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(name_of_drug.value_counts(), bins=160)
plt.yscale('log', nonpositive='clip')
plt.title('Log-Histogram of question appearance counts')
plt.xlabel('Number of occurences of drug name')
plt.ylabel('Number of drugs')
print ('Maximum number of times a single question is repeated: {}\n'.format(max(name_of_drug.value_counts()))) 

In [ ]:
use_case_drug = pd.Series(drug_train_data['use_case_for_drug'].tolist())
unique_use_case_drugs = len(np.unique(use_case_drug))
print ('Total number of  Unique Questions are: {}\n'.format(unique_use_case_drugs))

print ('Max number of times a single question is repeated: {}\n'.format(max(use_case_drug.value_counts()))) 

use_case_drug_vals=use_case_drug.value_counts()
print(use_case_drug_vals)

use_case_drug_vals=use_case_drug_vals.values

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(use_case_drug.value_counts(), bins=160)
plt.yscale('log', nonpositive='clip')
plt.title('Log-Histogram of question appearance counts')
plt.xlabel('Number of occurences of use case for drug')
plt.ylabel('Number of use case for drugs')
print ('Maximum number of times a single question is repeated: {}\n'.format(max(use_case_drug.value_counts()))) 

In [ ]:
# review_by_patient
drug_train_data['review_len'] = drug_train_data['review_by_patient'].str.len()
drug_train_data['review_n_words'] = drug_train_data['review_by_patient'].apply(lambda row: len(row.split(" ")))
drug_train_data.head(2) 

In [ ]:
# max_len_review = max(drug_train_data['review_len']) 
# min_len_review = min(drug_train_data['review_len']) 
# review_len_diff = max_len_review - min_len_review
# normal_review_len = 4 * (drug_train_data['review_len'] - min_len_review) / review_len_diff

# max_base_score = max(drug_train_data['base_score']) 
# min_base_score = min(drug_train_data['base_score']) 
# base_score_diff = max_base_score - min_base_score
# normal_base_score = (drug_train_data['base_score'] - min_base_score) / base_score_diff

# sns.kdeplot(normal_review_len, color="#7C3794", label='review_len')
# sns.kdeplot(normal_base_score, color="#00ff40", label='base_score')
# plt.legend()

In [ ]:
# plt.figure(figsize=(20, 10))
# plt.scatter(drug_train_data["base_score"], drug_train_data["review_len"], s=1)

<h2> 3.4 Preprocessing of Text </h2>

- Preprocessing:
    - Removing html tags 
    - Removing Punctuations
    - Performing stemming
    - Removing Stopwords
    - Expanding contractions etc.

In [ ]:
# To get the results in 4 decemal points
SAFE_DIV = 0.0001 

STOP_WORDS = stopwords.words("english")
stopword = set(STOP_WORDS)
stopword.add("said")
stopword.add("br")
stopword.add(" ")
stopword.remove("not")
stopword.remove("no")


def preprocess(x):
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will").replace("asap", "as soon as possible")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    
    
    porter = PorterStemmer()
    pattern = re.compile('\W')
    
    if type(x) == type(''):
        x = re.sub(pattern, ' ', x)
    
    
    if type(x) == type(''):
        x = porter.stem(x)
        example1 = BeautifulSoup(x)
        x = example1.get_text()
               
    return x
preprocess(drug_train_data['review_by_patient'][10091])

In [ ]:
drug_train_data["review_by_patient_filter"] = drug_train_data['review_by_patient'].fillna("").apply(preprocess)
drug_train_data["review_by_patient_unique_word"] = drug_train_data['review_by_patient_filter'].apply(lambda row: list(set([word for word in row.split() if word not in stopword])))
drug_train_data.head(2)

In [ ]:
drug_train_data["len_review_by_patient_unique_word"] = [len(ele) for ele in list(drug_train_data["review_by_patient_unique_word"])]
# plt.figure(figsize=(40, 20))
# plt.scatter(drug_train_data["base_score"], [len(ele) for ele in list(drug_train_data["review_by_patient_unique_word"])], s=1)

In [ ]:
# sns.FacetGrid(drug_train_data[["len_review_by_patient_unique_word", "effectiveness_rating"]], hue="effectiveness_rating", height=5).map(sns.distplot, "len_review_by_patient_unique_word").add_legend()
# plt.show()

In [ ]:
for i in range(0, 11):
    print(f'rating {i} mean words --> {drug_train_data[drug_train_data["effectiveness_rating"] == i].number_of_times_prescribed.mean()}')

In [ ]:
for i in range(0, 11):
    print(f'rating {i} mean words --> {drug_train_data[drug_train_data["effectiveness_rating"] == i].review_len.mean()}')

In [ ]:
for i in range(0, 11):
    print(f'rating {i} mean words --> {drug_train_data[drug_train_data["effectiveness_rating"] == i].review_n_words.mean()}')

In [ ]:
for i in range(0, 11):
    print(f'rating {i} mean words --> {drug_train_data[drug_train_data["effectiveness_rating"] == i].len_review_by_patient_unique_word.mean()}')

In [2]:
!python -m spacy download en_core_web_lg

2022-12-23 10:28:03.325489: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-12-23 10:28:03.325528: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-23 10:28:10.633656: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-12-23 10:28:10.633684: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-23 10:28:10.638513: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: DESKTOP-17MMCVT
2022-12-23 10:28:10.638593: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: DESKTOP-17MMCVT
ERROR: Exception:
Traceback (most recent call last):
  File "E:\python\anaconda3\lib\site-packages\pip\_ve

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
# !python -m spacy download en_core_web_lg
# !python -m spacy download en_core_web_sm
# !python -m spacy download en_core_web_md
!python -m spacy download en

In [ ]:
lst_review = list(drug_train_data["review_by_patient"])
len(lst_review)

In [ ]:
tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(lst_review)
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [ ]:
nlp = spacy.load('en_core_web_lg')
x=nlp('man')
len(x.vector)

In [ ]:
vecs2 = []
for qu2 in tqdm(list(df['question2'])):
    doc2 = nlp(qu2) 
    mean_vec1 = np.zeros([len(doc1), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df['q2_feats_m'] = list(vecs2)

In [ ]:
arr1 = np.array([-1.78585485e-01,  1.57477319e-01,  7.05434978e-02, -9.44870859e-02,
       -1.55617163e-01, -3.96468788e-01,  3.25957388e-01,  4.15079117e-01,
       -1.79139793e-01, -9.25561786e-02,  6.36538565e-02, -8.32231939e-02,
       -4.11425941e-02,  3.82984132e-01, -1.99536353e-01,  3.66756707e-01,
        2.13709503e-01,  4.75003004e-01,  3.26888442e-01,  2.96676099e-01,
        4.56181206e-02, -3.91842276e-01,  3.52340609e-01,  5.32376813e-03,
        7.10509121e-02,  3.00839730e-02,  1.57515138e-01, -1.94057435e-01,
        1.43760443e-03, -4.88332808e-02,  1.56052887e-01,  4.79005098e-01,
       -2.00121433e-01, -3.78197789e-01,  2.43874758e-01,  1.73838735e-01,
       -6.52950257e-02,  9.94619951e-02, -3.19469392e-01, -4.02338877e-02,
       -3.15540880e-01,  8.14640895e-02, -1.09709233e-01,  1.66549534e-02,
       -9.42767859e-02,  3.26686613e-02, -3.09479046e+00, -1.81728035e-01,
       -1.74628407e-01, -3.74800593e-01,  2.42744520e-01, -2.44913802e-01,
       -1.14570558e-01,  1.93448409e-01,  4.29629326e-01,  4.88099217e-01,
       -5.35073996e-01,  1.95371389e-01, -5.97972810e-01,  6.08351380e-02,
        3.58131111e-01, -1.04662441e-01, -4.69437778e-01, -7.97884911e-02,
        3.73414569e-02, -2.10961387e-01, -2.20836908e-01,  3.78496796e-01,
       -5.12835085e-01,  7.80248523e-01, -6.56149983e-01, -2.60785103e-01,
        1.10963657e-01, -3.68746929e-02, -2.07240209e-01,  2.07710385e-01,
        1.28326595e-01,  1.99494913e-01, -3.04864764e-01, -1.67069823e-01,
       -2.22333074e-02,  4.36324596e-01,  5.43525368e-02, -1.13456845e-02,
       -1.11501746e-01,  3.40306491e-01, -3.34517002e-01,  1.77581251e-01,
        7.59852901e-02,  4.30442452e-01,  3.07146877e-01,  7.73947388e-02,
        9.03457329e-02, -8.98420364e-02,  1.69031009e-01, -4.33041811e-01,
        2.25272164e-01,  1.30627930e-01,  1.52320176e-01,  4.36731726e-01,
        6.56451166e-01, -9.50943530e-02,  2.31117576e-01, -7.49166131e-01,
        4.85159874e-01, -2.16684654e-01,  6.33629709e-02, -2.83936858e-01,
        2.06875548e-01, -1.48586440e+00,  3.03163193e-02,  4.47456062e-01,
       -2.40944937e-01, -4.29122567e-01, -5.08502722e-01, -7.55460858e-02,
       -4.46311012e-02,  2.33831555e-01,  7.86455199e-02,  2.19635993e-01,
       -3.18726420e-01,  4.43274051e-01, -1.20752759e-01,  1.99196726e-01,
        1.75701641e-02,  6.79453254e-01, -3.33665788e-01, -3.46388102e-01,
        4.86138463e-02,  6.60473943e-01,  2.70166188e-01,  2.77149945e-01,
        3.61223519e-02, -1.82784230e-01, -3.59924912e-01,  5.73625207e-01,
        1.11215420e-01, -1.12384468e-01, -1.27173781e-01,  1.06251404e-01,
       -1.11925557e-01, -2.66250372e-01, -2.51601052e+00,  3.72133374e-01,
        4.62335259e-01,  9.18090641e-02, -2.12522000e-01,  2.40183949e-01,
       -1.33245945e-01,  2.66711593e-01,  2.81776208e-02, -1.04351670e-01,
       -1.34206802e-01, -7.87169710e-02, -4.50378731e-02,  3.08842838e-01,
       -8.66437316e-01,  7.42827803e-02, -1.43136650e-01, -1.04572922e-01,
        2.67032027e-01, -6.37553111e-02, -2.17727080e-01,  2.35526934e-01,
        1.12510100e-01,  3.96340489e-01,  4.43402231e-02,  2.68895596e-01,
        1.53070726e-02, -1.72325820e-01, -2.10673958e-01, -1.09551728e-01,
        9.31565702e-01, -5.20192444e-01,  5.03991246e-01, -2.45620742e-01,
       -2.07773030e-01,  4.74270672e-01,  2.19998688e-01, -2.94020563e-01,
       -4.18687880e-01,  7.26718903e-01, -2.03840107e-01, -1.41456008e-01,
        5.41512184e-02, -1.33927494e-01,  6.58535540e-01,  2.45598797e-02,
        5.64650707e-02,  1.31496161e-01, -2.94756949e-01, -3.68722707e-01,
        4.24532667e-02, -1.63967982e-01,  6.69974208e-01, -4.85353991e-02,
       -1.81698173e-01, -1.59758151e-01, -1.66436538e-01, -1.38218664e-02,
       -1.58954799e-01,  1.94022000e-01,  1.84793919e-01,  6.27244189e-02,
       -3.74134988e-01,  3.09919405e+00,  1.38984680e-01, -1.26568913e-01,
        3.84287328e-01,  7.54782200e-01,  3.69236991e-02, -6.67319745e-02,
        4.07265276e-02, -8.74960944e-02,  2.16343790e-01,  2.31922880e-01,
        4.73663568e-01,  1.72043517e-01,  1.09870344e-01, -2.15359122e-01,
        1.47773772e-01, -5.44703677e-02, -1.63218409e-01,  6.20274067e-01,
        2.46201903e-02,  4.53645080e-01, -2.36396074e-01, -6.46556094e-02,
        1.76679939e-01, -1.38106787e+00,  4.06075001e-01, -4.65069935e-02,
        4.56188858e-01,  9.44129154e-02, -2.92724669e-01, -1.41476244e-01,
        6.89983889e-02, -2.99979597e-01, -1.06438883e-01, -1.91148058e-01,
       -1.97263986e-01,  2.74489939e-01,  2.00958639e-01,  1.27398580e-01,
       -1.31507069e-01,  1.71426013e-01, -5.99647835e-02, -2.81197309e-01,
       -6.72750920e-02, -9.20163095e-02,  1.14901595e-01,  1.29807994e-01,
        1.04418263e-01, -1.49112433e-01,  3.23003352e-01,  2.07921460e-01,
       -1.40071124e-01,  4.19391654e-02, -2.22299442e-01, -2.01070189e-01,
       -2.93633044e-01,  6.35917708e-02,  3.84784117e-03,  4.11615431e-01,
       -5.25972188e-01, -3.28684926e-01, -2.03504771e-01,  1.16835535e-02,
       -1.88837573e-01, -4.40943539e-02,  1.35757104e-02, -3.13318551e-01,
       -1.00122952e+00, -2.13271594e+00,  1.71365459e-02, -2.58547157e-01,
        1.04791731e-01,  2.66524881e-01, -1.10024363e-01, -5.27077578e-02,
        4.06972468e-01,  6.23354435e-01,  4.92507480e-02,  7.60561824e-01,
        2.56005943e-01, -3.45146358e-01,  9.89977643e-02, -2.49707788e-01,
        3.25770155e-02, -3.22994702e-02, -5.35687447e-01,  9.66915935e-02,
       -4.26726937e-01, -2.89204389e-01,  2.30049103e-01,  2.29605176e-02,
        2.49943256e-01,  1.96090221e-01,  1.81873381e-01, -3.68742049e-01,
        2.32710913e-02,  2.77605921e-01,  3.77970278e-01,  1.96092293e-01,
       -9.11547542e-02,  1.53488368e-01, -4.61188972e-01, -3.01808506e-01,
       -3.61204386e+00, -1.38310730e-01, -2.20187619e-01, -4.96620446e-01,
       -9.02001187e-02,  4.30062562e-02,  4.53471720e-01, -1.60062909e-01,
       -3.55108678e-01, -1.55201554e-02,  1.92107335e-02, -1.32532761e-01,
        3.42323095e-01,  4.98508960e-02,  4.02669013e-01,  3.58660936e-01,
        6.64951682e-01, -2.73034722e-01,  4.48142827e-01,  6.87834024e-01,
        1.64970040e-01, -1.24821000e-01,  3.09652258e-02,  6.09567836e-02,
        2.62018323e-01,  4.80348229e-01, -5.04801214e-01,  1.89316180e-02,
        8.61565247e-02, -2.08694428e-01,  1.97748303e-01, -4.32038724e-01,
       -2.14172557e-01, -1.77567303e-02, -3.95232856e-01, -9.17381272e-02,
       -1.87501550e-01,  6.64074123e-02,  3.22061241e-01, -1.15241751e-01,
       -2.86419481e-01,  5.14847815e-01,  2.18125761e-01, -1.60580307e-01,
        7.68804729e-01,  1.85636878e-01,  2.01396674e-01,  1.01520665e-01,
       -2.89083004e-01,  6.22533083e-01,  2.13385046e-01, -2.69795239e-01,
        8.75692785e-01, -1.02372572e-01, -2.39857674e-01, -8.75274912e-02,
        4.39553618e-01,  1.93445131e-01, -6.08878285e-02,  1.31628156e-01,
        3.26126993e-01, -2.63141602e-01, -2.09245279e-01, -4.07686293e-01,
        9.11217183e-02, -5.89886785e-01,  4.98137414e-01, -4.35920000e-01,
        1.04751252e-02,  1.93697304e-01, -2.17374399e-01,  3.99149925e-01,
       -3.26753110e-01, -7.76505709e-01, -9.62390900e-02, -1.68129131e-01,
       -5.55748045e-02,  3.23731661e-01, -7.69721866e-02,  4.25359488e-01,
       -5.50715685e-01, -1.52445398e-03, -4.08857286e-01,  4.44858551e-01,
       -1.73323452e-01,  1.61044359e-01, -1.36966988e-01,  3.29537719e-01,
       -6.27062738e-01, -4.28624153e-02,  8.77703130e-02,  3.87711138e-01,
        4.45994288e-01,  1.56538397e-01, -2.99938053e-01, -1.52427852e-02,
        3.97159725e-01, -6.54679954e-01,  2.16623902e-01, -3.53115767e-01,
        8.04682374e-02,  4.38643172e-02,  3.30340058e-01, -3.66500050e-01,
       -2.45856866e-01, -2.40360618e-01, -2.78139323e-01, -3.28908674e-02,
        4.52815555e-02, -1.39600098e-01,  1.25325784e-01, -1.89531147e-01,
        8.21577013e-02,  2.79966354e-01,  7.58138597e-01, -1.42798036e-01,
       -1.60481874e-03,  3.57777655e-01,  1.47333369e-01, -6.13354035e-02,
        1.79496869e-01, -2.38619745e-02, -2.63873875e-01, -2.90113419e-01,
       -4.24245745e-03, -2.72856057e-01, -9.81459841e-02, -2.57121056e-01,
       -2.08448172e-01,  4.87759262e-01,  1.54264718e-01,  8.30523446e-02,
       -4.05403614e-01,  3.01687606e-02, -2.13147908e-01, -3.13316792e-01,
       -1.12057962e-02,  3.03139538e-01,  3.40467036e-01,  9.49243158e-02,
        4.84702140e-01, -2.81111225e-02, -2.29207411e-01,  3.30829740e-01,
       -1.63027391e-01,  4.39656258e-01, -1.76206887e-01, -1.40993461e-01,
       -2.74910331e-01,  2.71663904e-01, -4.34911251e-02, -1.16522595e-01,
       -3.71948540e-01, -3.86367679e-01,  4.56152380e-01,  3.79977673e-01,
       -2.02422231e-01,  1.53720155e-01, -3.27459753e-01, -6.36289597e-01,
        2.04745695e-01, -6.06635585e-02, -1.86270404e+00,  5.13879776e-01,
        1.64138228e-02,  9.60999057e-02, -9.41863284e-02, -1.13554165e-01,
        1.14640892e-01,  6.56561255e-01,  2.41455197e-01, -9.79864299e-02,
       -5.56795895e-01, -1.59727842e-01,  5.47385998e-02,  2.73323655e-01,
       -6.97492361e-02, -7.07473829e-02, -4.06489879e-01, -4.36569005e-02,
       -1.45252869e-01, -1.89866811e-01, -6.14446700e-01,  3.90102774e-01,
        7.75060281e-02,  5.75042106e-02,  2.36010849e-01, -3.17493379e-01,
        5.28088808e-02,  7.80421495e-02,  1.47031739e-01,  5.90730250e-01,
       -6.48618788e-02, -2.73236901e-01, -1.04597718e-01, -4.67394561e-01,
        2.99266070e-01, -1.23662993e-01,  1.23696133e-01, -1.00500062e-01,
        6.05877340e-01,  6.67537034e-01, -3.18957180e-01,  3.40545893e-01,
        1.36671543e-01,  9.41669196e-02,  4.03697670e-01,  3.15705925e-01,
       -2.54409909e-01,  1.50968269e-01, -3.37513626e-01, -8.31855536e-02,
       -1.82415739e-01, -2.44417697e-01, -2.61873394e-01, -2.65761524e-01,
       -2.03441039e-01, -1.63466752e-01, -4.22960430e-01,  4.35326219e-01,
       -9.79961008e-02, -4.22616825e-02,  1.92254521e-02, -1.49543628e-01,
       -1.96997643e-01,  9.44274962e-02,  1.50467515e-01, -7.91059554e-01,
        2.42577612e-01,  3.29445750e-02, -1.29345804e-01,  1.84590578e-01,
        5.53250432e-01,  1.32597938e-01, -3.35771680e-01,  4.57832068e-01,
       -3.73100899e-02,  2.12995604e-01,  5.70987426e-02,  8.84095579e-02,
       -4.72625703e-01,  7.08459094e-02, -4.90241349e-02, -6.78504348e-01,
       -3.90968502e-01,  4.69217837e-01, -9.16147754e-02, -2.91010179e-02,
       -4.21310365e-01,  2.67117471e-01,  2.79747486e-01, -4.04535308e-02,
        3.95763330e-02, -5.20985723e-01, -2.55432576e-02,  4.65565175e-02,
       -9.14442018e-02,  2.66722776e-02, -1.33391589e-01,  6.40659854e-02,
        1.78339541e-01,  1.78867400e-01, -1.83651745e-01,  4.66993988e-01,
        3.84450823e-01,  2.12169960e-01,  4.87828180e-02,  1.33566499e-01,
        2.29756132e-01, -2.55477190e-01, -1.37217402e-01, -1.54591158e-01,
        9.00893211e-02, -3.50445285e-02,  2.04404861e-01,  4.01059747e-01,
        1.22957021e-01, -5.76993823e-01, -9.37057287e-02, -2.34698668e-01,
        1.45873487e+00,  2.25164518e-01,  8.47412944e-02, -5.70528544e-02,
        2.55481154e-01, -1.91265374e-01, -2.73182660e-01,  2.12907732e-01,
       -2.71648616e-01,  2.03765690e-01, -2.52506942e-01,  3.48273337e-01,
       -4.37393993e-01, -1.17386997e-01,  3.68586779e-01,  3.48050654e-01,
        8.55632871e-02, -1.46381319e-01, -3.68343711e-01, -2.36205280e-01,
       -2.87411660e-01,  1.85269676e-02,  3.70165825e-01,  1.48867503e-01,
       -1.37645841e-01,  4.45979923e-01,  1.43662304e-01, -2.86338687e-01,
        2.43107617e-01,  6.75095543e-02, -1.13206267e-01, -7.95914084e-02,
        8.00015330e-01,  4.52473044e-01, -4.61673081e-01, -2.92752385e-01,
        7.81234354e-02, -1.31819487e-01, -7.07301572e-02,  2.06201404e-01,
        3.59891415e-01, -4.29966122e-01,  6.29848421e-01, -3.85940909e-01,
       -8.90899152e-02,  5.40900230e-01, -8.36291760e-02, -3.10393214e-01,
        2.80933797e-01,  5.04332423e-01,  1.21395096e-01,  1.55635163e-01,
       -1.61031038e-01,  1.35347039e-01,  9.79719162e-02,  4.73171845e-03,
       -2.98989773e-01, -3.46274018e-01, -2.80406386e-01,  1.22619733e-01,
        6.31065071e-02,  3.29333782e-01,  3.90275210e-01,  3.00008729e-02,
        2.05134988e-01,  3.73953342e-01, -2.34413654e-01,  2.11343139e-01,
        3.40234131e-01, -2.79085696e-01,  1.54385820e-01,  7.38023639e-01,
        1.95696682e-01,  4.81828868e-01,  1.97307155e-01,  6.19039953e-01,
        6.76824510e-01,  1.13667205e-01, -5.00577152e-01, -1.18917441e+00,
        1.20900303e-01, -8.13509077e-02,  2.41672546e-02,  2.41864234e-01,
        4.35197204e-01,  3.62388968e-01, -1.53635100e-01, -2.04461530e-01,
       -3.06267560e-01,  4.23407823e-01,  5.27687110e-02,  1.95663035e-01,
       -2.78501570e-01, -6.15150109e-02, -3.67549121e-01,  4.42736059e-01,
       -2.69443989e-01, -3.65576297e-01, -1.09531358e-01, -5.06751910e-02,
        4.24663186e-01,  2.51884311e-01, -3.75782102e-01, -3.40543330e-01,
        4.51876044e-01, -2.23814070e-01, -4.87366080e-01,  2.29054391e-01,
       -4.67492230e-02,  4.43604887e-02,  3.00667733e-01,  4.79085073e-02,
       -2.57694006e-01, -3.58638555e-01,  3.12085338e-02, -1.77585080e-01,
        1.07200786e-01, -1.31780311e-01, -2.36209333e-02, -3.62601757e-01,
        3.11655998e-01, -3.92216623e-01,  1.23689272e-01, -4.99594286e-02,
        3.14061344e-02,  5.19580066e-01, -2.90836513e-01,  6.99612796e-01,
       -1.24776945e-01, -2.64312297e-01, -2.21475795e-01,  8.88104022e-01,
        1.05979092e-01,  7.63355196e-02,  4.86734323e-04, -1.06293708e-01,
        6.45845057e-03, -2.16941714e-01, -4.90010083e-01, -2.84463823e-01,
       -1.54776067e-01,  4.68855947e-02, -4.10257056e-02,  1.69675723e-02,
       -4.36892569e-01, -3.95191938e-01, -8.78178515e-03,  1.42297506e-01,
       -5.54722250e-01, -3.03702503e-01, -2.56756693e-01,  1.60002738e-01,
        1.90465301e-01,  1.33697987e-01, -3.02134395e-01,  3.47311974e-01,
        4.56263363e-01,  4.63207275e-01, -2.66119987e-02,  2.06031442e-01,
        1.90829039e-02, -2.75941491e-01, -2.14834526e-01,  1.03752404e-01,
       -4.81115150e+00, -4.35087204e-01, -3.40041667e-02, -3.30906749e-01,
       -1.06570333e-01, -5.01041591e-01, -1.01450011e-02, -4.76721823e-01,
       -3.85644212e-02, -6.44545406e-02,  6.75440505e-02,  2.54303902e-01,
       -2.61652470e-01, -1.70410335e-01,  2.83850372e-01,  1.95463106e-01],
      dtype=float32)